In [39]:
import os

### Select Image
image_file = "pic5.jpg"
image_name = os.path.splitext(image_file)[0]
image_path = "sample_pictures/" + image_file

In [40]:
import requests
import json
from PIL import Image
import base64

http_url = 'https://api-us.faceplusplus.com/facepp/v3/detect'
key = "7AUVsIF0R8T2YYZmrfe5-4r9FdumOFYc"
secret = "0TF9Xcz_9RVi10q_2Y6SqKdkiM8Fmlze"
with open(image_path, "rb") as img_file:
    img_string = base64.b64encode(img_file.read())
response = requests.post(
    'https://api-us.faceplusplus.com/facepp/v3/detect',
    {
        'api_key': key,
        'api_secret': secret,
        'image_base64': img_string,
        'return_landmark': 1,
        'return_attributes': 
'headpose,eyestatus,facequality,mouthstatus,eyegaze,blur,emotion'
    }
)
data = response.json()
print(data)
json_file=os.path.splitext(image_name)[0] + ".json"
with open('api_results/' + json_file, 'w') as outfile:
    json.dump(data, outfile)

{'request_id': '1606954914,1bf3daa2-9b3a-4b48-9ed1-cb6d53fab3d4', 'time_used': 68, 'faces': [{'face_token': '6135cc00d1b6b71411e43e4a30a43714', 'face_rectangle': {'top': 156, 'left': 311, 'width': 202, 'height': 202}, 'landmark': {'contour_chin': {'x': 453, 'y': 352}, 'contour_left1': {'x': 310, 'y': 213}, 'contour_left2': {'x': 314, 'y': 237}, 'contour_left3': {'x': 321, 'y': 260}, 'contour_left4': {'x': 330, 'y': 282}, 'contour_left5': {'x': 343, 'y': 303}, 'contour_left6': {'x': 359, 'y': 322}, 'contour_left7': {'x': 379, 'y': 336}, 'contour_left8': {'x': 401, 'y': 347}, 'contour_left9': {'x': 426, 'y': 354}, 'contour_right1': {'x': 490, 'y': 177}, 'contour_right2': {'x': 494, 'y': 197}, 'contour_right3': {'x': 497, 'y': 218}, 'contour_right4': {'x': 497, 'y': 239}, 'contour_right5': {'x': 495, 'y': 260}, 'contour_right6': {'x': 492, 'y': 281}, 'contour_right7': {'x': 487, 'y': 301}, 'contour_right8': {'x': 481, 'y': 321}, 'contour_right9': {'x': 471, 'y': 339}, 'left_eye_bottom': {

In [41]:
import numpy as np
import cv2

### p1 left point, p2 right point
def point_on_line (p1, p2, ratio):
    x = p1[0] + ratio*(p2[0]-p1[0])
    y = p1[1] + ratio*(p2[1]-p1[1])
    return (int(x), int(y))

def get_ratio (p1, p2, p3):
    return (p3[0]-p1[0])/(p2[0]-p1[0])

def closest_point_on_line(p1, p2, p3):
    x1, y1 = p1
    x2, y2 = p2
    x3, y3 = p3
    dx, dy = x2-x1, y2-y1
    det = dx*dx + dy*dy
    a = (dy*(y3-y1)+dx*(x3-x1))/det
    return (int(x1+a*dx), int(y1+a*dy))

### Create map of offset from reference point
def offset_map (landmarks, ref_pt):
    off_map = {}
    for feature in landmarks.keys():
        pt= (landmarks[feature]['x'], landmarks[feature]['y'])
        off_map[feature] = (pt[0] - ref_pt[0], pt[1] - ref_pt[1])
    return off_map

class HumanFace:
    landmarks = {}
    def __init__(self, face):
        self.landmarks = face['landmark']  

    def findCorners(self, points_dict):
        left, top = 2147483646, 2147483646
        right, bottom = 0, 0
        for feat, coord in points_dict.items():
            if coord['x'] < left: 
                left = coord['x']
            if coord['x'] > right: 
                right = coord['x']
            if coord['y'] < top: 
                top = coord['y']
            if coord['y'] > bottom: 
                bottom = coord['y']
        return left, right, bottom, top

    def get_dict(self, keyword):
        key_list = []
        for feature in self.landmarks.keys():
            if keyword in feature:
                key_list.append(feature)
        feature_dict = {k:v for k,v in self.landmarks.items() if k in key_list}
        return feature_dict

    def get_object_region(self, body_part):
        body_dict = self.get_dict(body_part)
        left, right, bottom, top = self.findCorners(body_dict)
        region_dict = {'left': left + 1, 'right': right + 1, 'top': top, 'bottom': bottom + 10}
        return region_dict
    
        
### Create map of offset from reference point
def get_offset (points, ref_pt):
    offsets = []
    for pt in points:
        offsets.append((pt[0] - ref_pt[0], pt[1] - ref_pt[1]))
    return offsets

def apply_inpaint(img, top, bottom, left, right, window_size):
    mask = img.copy()
    mask[:,:] = 0
    mask[top-5:bottom+5,left-5:right+5] = 255
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
    dst = cv2.inpaint(img,mask,25,cv2.INPAINT_TELEA)
    
    blurred_mask = img.copy()
    blurred_mask[:,:] = 0
    blurred_mask[top-5:bottom+5,left-5:right+5]=255
    blurred_mask = cv2.GaussianBlur(blurred_mask,(window_size,window_size),0)
    blurred_mask = blurred_mask.astype('float') / 255.
    img = img.astype('float') / 255.
    dst = dst.astype('float') / 255.
    img = img * (1 - blurred_mask) + dst * blurred_mask
    img = (img * 255).astype('uint8')
    return img
    


In [42]:
json_file=os.path.splitext(image_name)[0] + ".json"
with open('api_results/' + json_file) as f:
    data = json.load(f)
faces = data['faces']
img = cv2.imread(image_path)
og_img = img.copy()

regions = ['nose', 'left_eye', 'right_eye', 'mouth']

# Inpaint facial regions
for region in regions:
    for face in faces:
        subject = HumanFace(face) 
        feature_region = subject.get_object_region(region)
        point1 = (feature_region['left'], feature_region['top'])
        point2 = (feature_region['right'], feature_region['bottom'])

        ### Apply Inpaint
        img = apply_inpaint(img, point1[1], point2[1], point1[0], point2[0], 67)
    
### Inpaint Middle Eye Region
for face in faces:
    subject = HumanFace(face) 
    landmarks = subject.landmarks
    right_eye_region = subject.get_object_region('right_eye')
    left_eye_region = subject.get_object_region('left_eye')
    top = min(left_eye_region['top'], right_eye_region['top'])
    bottom = max(left_eye_region['bottom'], right_eye_region['bottom'])
    left = left_eye_region['right']
    right = right_eye_region['left']
    
    ### Apply Inpaint
    img = apply_inpaint(img, top, bottom, left, right, 99)

### Shift Left Eye
for face in faces:
    subject = HumanFace(face) 
    landmarks = subject.landmarks
    left_eye_region = subject.get_object_region('left_eye')
    point1 = (left_eye_region['left'], left_eye_region['top'])
    point2 = (left_eye_region['right'], left_eye_region['bottom'])
    ### point1[0] : left | point1[1] : top | point2[0] : right | point2[1] : bottom
    points = [point1, point2]
    
    start = 'contour_left1'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right1'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'left_eye_pupil'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
    new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
    
    offsets = get_offset (points, ref_point)
    offset1 = offsets[0]
    new_point1 = (new_ref[0] + offset1[0], new_ref[1] + offset1[1])
    offset2 = offsets[1]
    new_point2 = (new_ref[0] + offset2[0], new_ref[1] + offset2[1])
    left_eye = og_img[point1[1]:point2[1], point1[0]:point2[0]]
    left_eye = left_eye.copy()
    
    mask = 255 * np.ones(left_eye.shape, left_eye.dtype)
    center = (int((new_point1[0]+new_point2[0])/2), int((new_point1[1]+new_point2[1])/2))
    img = cv2.seamlessClone(left_eye, img, mask, center, cv2.NORMAL_CLONE)

### Shift Right Eye
for face in faces:
    subject = HumanFace(face) 
    landmarks = subject.landmarks
    right_eye_region = subject.get_object_region('right_eye')
    point1 = (right_eye_region['left'], right_eye_region['top'])
    point2 = (right_eye_region['right'], right_eye_region['bottom'])
    ### point1[0] : left | point1[1] : top | point2[0] : right | point2[1] : bottom
    points = [point1, point2]
    
    start = 'contour_left1'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right1'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'right_eye_pupil'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
    new_start = 'contour_left3'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right3'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
    
    offsets = get_offset (points, ref_point)
    offset1 = offsets[0]
    new_point1 = (new_ref[0] + offset1[0], new_ref[1] + offset1[1])
    offset2 = offsets[1]
    new_point2 = (new_ref[0] + offset2[0], new_ref[1] + offset2[1])
    right_eye = og_img[point1[1]:point2[1], point1[0]:point2[0]]
    right_eye = right_eye.copy()

    mask = 255 * np.ones(right_eye.shape, right_eye.dtype)
    center = (int((new_point1[0]+new_point2[0])/2), int((new_point1[1]+new_point2[1])/2))
    img = cv2.seamlessClone(right_eye, img, mask, center, cv2.NORMAL_CLONE)
    
### Shift Mouth
for face in faces:
    subject = HumanFace(face) 
    landmarks = subject.landmarks
    mouth_region = subject.get_object_region('mouth')
    point1 = (mouth_region['left'] - 10, mouth_region['top'])
    point2 = (mouth_region['right'] + 10, mouth_region['bottom'])
    ### point1[0] : left | point1[1] : top | point2[0] : right | point2[1] : bottom
    points = [point1, point2]
    
    start = 'contour_left5'; start_point = (landmarks[start]['x'], landmarks[start]['y'])
    end = 'contour_right5'; end_point = (landmarks[end]['x'], landmarks[end]['y'])
    ref = 'mouth_upper_lip_top'; ref_point = (landmarks[ref]['x'], landmarks[ref]['y'])
    new_ref = closest_point_on_line(start_point, end_point, ref_point)
    
    new_start = 'contour_left4'; new_start_point = (landmarks[new_start]['x'], landmarks[new_start]['y'])
    new_end = 'contour_right4'; new_end_point = (landmarks[new_end]['x'], landmarks[new_end]['y'])
    new_ref =  point_on_line (new_start_point, new_end_point, get_ratio(start_point, end_point, new_ref))
    
    offsets = get_offset (points, ref_point)
    offset1 = offsets[0]
    new_point1 = (new_ref[0] + offset1[0], new_ref[1] + offset1[1])
    offset2 = offsets[1]
    new_point2 = (new_ref[0] + offset2[0], new_ref[1] + offset2[1])
    mouth = og_img[point1[1]:point2[1], point1[0]:point2[0]]
    mouth = mouth.copy()
    
    mask = 255 * np.ones(mouth.shape, mouth.dtype)
    center = (int((new_point1[0]+new_point2[0])/2), int((new_point1[1]+new_point2[1])/2))
    img = cv2.seamlessClone(mouth, img, mask, center, cv2.NORMAL_CLONE)
    
  
cv2.imwrite('sample_outputs/' + image_name + "_output.jpg", img) 
cv2.imshow('image',img)
cv2.waitKey(0)


-1